In [2]:
import datetime

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [7]:
#data_dir = 'drive/My Drive/'
data_dir = 'data/'
train_lang = 'en'

In [9]:
class DatasetSeq(Dataset):
    def __init__(self, data_dir, train_lang='en'):
	#open file
        with open(data_dir + train_lang + '.train', 'r', encoding='utf-8') as f:
            train = f.read().split('\n\n')

        # delete extra tag markup
        train = [x for x in train if not '_ ' in x]
	    #init vocabs of tokens for encoding {<str> token: <int> id}
        self.target_vocab = {'<pad>': 0} # {p: 1, a: 2, r: 3, pu: 4}
        self.word_vocab = {'<pad>': 0} # {cat: 1, sat: 2, on: 3, mat: 4, '.': 5}
        self.char_vocab = {'<pad>': 0} # {c: 1, a: 2, t: 3, ' ': 4, s: 5}
	    
        # Cat sat on mat. -> [1, 2, 3, 4, 5]
        # p    a  r  p pu -> [1, 2, 3, 1, 4]
        # chars  -> [1, 2, 3, 4, 5, 2, 3, 4]

	    #init encoded sequences lists (processed data)
        self.encoded_sequences = []
        self.encoded_targets = []
        self.encoded_char_sequences = []
        # n=1 because first value is padding
        n_word = 1
        n_target = 1
        n_char = 1
        for line in train:
            sequence = []
            target = []
            chars = []
            for item in line.split('\n'):
                if item != '':
                    word, label = item.split(' ')

                    if self.word_vocab.get(word) is None:
                        self.word_vocab[word] = n_word
                        n_word += 1
                    if self.target_vocab.get(label) is None:
                        self.target_vocab[label] = n_target
                        n_target += 1
                    for char in word:
                        if self.char_vocab.get(char) is None:
                            self.char_vocab[char] = n_char
                            n_char += 1
                    sequence.append(self.word_vocab[word])
                    target.append(self.target_vocab[label])
                    chars.append([self.char_vocab[char] for char in word])
            self.encoded_sequences.append(sequence)
            self.encoded_targets.append(target)
            self.encoded_char_sequences.append(chars)

    def __len__(self):
        return len(self.encoded_sequences)

    def __getitem__(self, index):
        return {
            'data': self.encoded_sequences[index], # [1, 2, 3, 4, 6] len=5
            'char': self.encoded_char_sequences[index],# [[1,2,3], [4,5], [1,2], [2,6,5,4], []] len=5
            'target': self.encoded_targets[index], # [1, 2, 3, 4, 6] len=5
        }

In [10]:
dataset = DatasetSeq(data_dir)

In [ ]:
#padding
# seq1 = [1, 2, 3, 4]
# seq2 = [9, 7, 6, 4, 3, 7, 5]
# pad seq1 equal seq2
# seq1 = [1, 2, 3, 4, 0, 0, 0]
# concat(seq1, seq2) [[1, 2, 3, 4, 0, 0, 0],
#                     [9, 7, 6, 4, 3, 7, 5]]

In [13]:
def collate_fn(batch):
    data = []
    target = []
    for item in batch:
        data.append(torch.as_tensor(item['data']))
        target.append(torch.as_tensor(item['target']))
    data = pad_sequence(data, batch_first=True, padding_value=0)
    target = pad_sequence(target, batch_first=True, padding_value=0)

    return {'data': data, 'target': target}

In [16]:
#hyper params
vocab_size = len(dataset.word_vocab) + 1
n_classes = len(dataset.target_vocab) + 1
n_chars = len(dataset.char_vocab) + 1
#TODO try to use other model parameters
emb_dim = 256
hidden = 256
n_epochs = 10
batch_size = 64
cuda_device = 0
batch_size = 100
if torch.cuda.is_available():
    device = f'cuda:{cuda_device}'
else:
    device = 'cpu'

In [28]:
rus = {'PROPN': 'имя собственное',
 'PUNCT': 'пунктуация',
 'ADJ': 'прилагательное',
 'NOUN': 'существительное',
 'VERB': 'глагол',
 'DET': 'определитель',
 'ADP': 'послелог',
 'AUX': 'вспомогательный глагол',
 'PRON': 'местоимение',
 'PART': 'частица ',
 'SCONJ': 'союз подчинительный',
 'NUM': 'числительное',
 'ADV': 'наречие',
 'CCONJ': 'союз сочинительный',
 'X': 'другое',
 'INTJ': 'междометие',
 'SYM': 'символ'}

# RNN

In [19]:
class RNNPredictor(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, n_classes):
        super().__init__()
        self.word_emb = nn.Embedding(vocab_size, emb_dim)
        #TODO try to use other RNN archicetures, f.e. RNN and LSTM
        self.rnn = nn.RNN(emb_dim, hidden_dim, batch_first=True)
        self.clf = nn.Linear(hidden_dim, n_classes)
        self.do = nn.Dropout(0.1)

    def forward(self, x):
        emb = self.word_emb(x) # B x T x Emb_dim
        hidden, _ = self.rnn(emb) # B x T x Hid, B x 1 x Hid
        hidden = self.do(hidden)
        pred = self.clf(hidden) # B x T x N_classes

        return pred

In [22]:
model_RNN = RNNPredictor(vocab_size, emb_dim, hidden, n_classes).to(device)
model_RNN.train()
optim = torch.optim.Adam(model_RNN.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

In [23]:
start = datetime.datetime.now()
for epoch in range(n_epochs):
    dataloader = DataLoader(dataset,
                            batch_size,
                            shuffle=True,
                            collate_fn=collate_fn,
                            drop_last = True,
                            )
    for i, batch in enumerate(dataloader):
        optim.zero_grad()

        predict = model_RNN(batch['data'].to(device))
        loss = loss_func(predict.view(-1, n_classes),
                         batch['target'].to(device).view(-1),
                         )
        loss.backward()
        optim.step()
        if i % 100 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')

    torch.save(model_RNN.state_dict(), f'./data/chkpt/rnn_chkpt_{epoch}.pth')
end = datetime.datetime.now() - start
print(end)

epoch: 0, step: 0, loss: 3.1829702854156494
epoch: 0, step: 100, loss: 0.19392119348049164
epoch: 0, step: 200, loss: 0.23536968231201172
epoch: 1, step: 0, loss: 0.17857904732227325
epoch: 1, step: 100, loss: 0.1774771362543106
epoch: 1, step: 200, loss: 0.15020506083965302
epoch: 2, step: 0, loss: 0.15298433601856232
epoch: 2, step: 100, loss: 0.11475085467100143
epoch: 2, step: 200, loss: 0.11459320783615112
epoch: 3, step: 0, loss: 0.089115209877491
epoch: 3, step: 100, loss: 0.09685000777244568
epoch: 3, step: 200, loss: 0.07078645378351212
epoch: 4, step: 0, loss: 0.11050165444612503
epoch: 4, step: 100, loss: 0.09401128441095352
epoch: 4, step: 200, loss: 0.07012433558702469
epoch: 5, step: 0, loss: 0.04284331202507019
epoch: 5, step: 100, loss: 0.059045128524303436
epoch: 5, step: 200, loss: 0.07682736963033676
epoch: 6, step: 0, loss: 0.06254022568464279
epoch: 6, step: 100, loss: 0.05435020476579666
epoch: 6, step: 200, loss: 0.06147751584649086
epoch: 7, step: 0, loss: 0.044

In [51]:
#example
phrase = 'He ran quickly after the red bus and caught it'
words = phrase.split(' ')
tokens = [dataset.word_vocab[w] for w in words]

start = datetime.datetime.now()
with torch.no_grad():
    model_RNN.eval()
    predict = model_RNN(torch.tensor(tokens).unsqueeze(0).to(device)) # 1 x T x N_classes
    labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
    end = datetime.datetime.now() - start

target_labels = list(dataset.target_vocab.keys())
print([rus[target_labels[l]] for l in labels])
print(end)

['местоимение', 'глагол', 'наречие', 'послелог', 'определитель', 'прилагательное', 'существительное', 'союз сочинительный', 'глагол', 'местоимение']
0:00:00.001998


In [50]:
#example 2
phrase = 'My mother seldom watches TV in the living-room , because she has little time'
words = phrase.split(' ')
tokens = [dataset.word_vocab[w] for w in words]

start = datetime.datetime.now()
with torch.no_grad():
    model_RNN.eval()
    predict = model_RNN(torch.tensor(tokens).unsqueeze(0).to(device)) # 1 x T x N_classes
    labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
    end = datetime.datetime.now() - start

target_labels = list(dataset.target_vocab.keys())
print([rus[target_labels[l]] for l in labels])
print(end)

['местоимение', 'существительное', 'наречие', 'глагол', 'существительное', 'послелог', 'определитель', 'существительное', 'пунктуация', 'союз подчинительный', 'местоимение', 'вспомогательный глагол', 'прилагательное', 'существительное']
0:00:00.001996


# LSTM

In [52]:
class LSTMpredictor(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, n_classes):
        super().__init__()
        self.word_emb = nn.Embedding(vocab_size, emb_dim)
        #TODO try to use other RNN archicetures, f.e. RNN and LSTM
        self.rnn = nn.LSTM(emb_dim, hidden_dim, batch_first=True)
        self.clf = nn.Linear(hidden_dim, n_classes)
        self.do = nn.Dropout(0.1)

    def forward(self, x):
        emb = self.word_emb(x) # B x T x Emb_dim
        hidden, _ = self.rnn(emb) # B x T x Hid, B x 1 x Hid
        hidden = self.do(hidden)
        pred = self.clf(hidden) # B x T x N_classes

        return pred

In [53]:
model_LSTM = LSTMpredictor(vocab_size, emb_dim, hidden, n_classes).to(device)
model_LSTM.train()
optim = torch.optim.Adam(model_LSTM.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

In [54]:
start = datetime.datetime.now()
for epoch in range(n_epochs):
    dataloader = DataLoader(dataset,
                            batch_size,
                            shuffle=True,
                            collate_fn=collate_fn,
                            drop_last = True,
                            )
    for i, batch in enumerate(dataloader):
        optim.zero_grad()

        predict = model_LSTM(batch['data'].to(device))
        loss = loss_func(predict.view(-1, n_classes),
                         batch['target'].to(device).view(-1),
                         )
        loss.backward()
        optim.step()
        if i % 100 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')

    torch.save(model_LSTM.state_dict(), f'./data/chkpt/lstm_chkpt_{epoch}.pth')
end = datetime.datetime.now() - start
print(end)

epoch: 0, step: 0, loss: 3.076673984527588
epoch: 0, step: 100, loss: 0.40494850277900696
epoch: 0, step: 200, loss: 0.2414265125989914
epoch: 1, step: 0, loss: 0.16726931929588318
epoch: 1, step: 100, loss: 0.1789942979812622
epoch: 1, step: 200, loss: 0.18894536793231964
epoch: 2, step: 0, loss: 0.08657508343458176
epoch: 2, step: 100, loss: 0.13518747687339783
epoch: 2, step: 200, loss: 0.07422366738319397
epoch: 3, step: 0, loss: 0.10222505778074265
epoch: 3, step: 100, loss: 0.08617609739303589
epoch: 3, step: 200, loss: 0.08567441999912262
epoch: 4, step: 0, loss: 0.06699670851230621
epoch: 4, step: 100, loss: 0.07051361352205276
epoch: 4, step: 200, loss: 0.08865643292665482
epoch: 5, step: 0, loss: 0.06983352452516556
epoch: 5, step: 100, loss: 0.04261178895831108
epoch: 5, step: 200, loss: 0.06174503266811371
epoch: 6, step: 0, loss: 0.06081746146082878
epoch: 6, step: 100, loss: 0.05795649439096451
epoch: 6, step: 200, loss: 0.054258719086647034
epoch: 7, step: 0, loss: 0.037

In [55]:
#example
phrase = 'He ran quickly after the red bus and caught it'
words = phrase.split(' ')
tokens = [dataset.word_vocab[w] for w in words]

start = datetime.datetime.now()
with torch.no_grad():
    model_LSTM.eval()
    predict = model_LSTM(torch.tensor(tokens).unsqueeze(0).to(device)) # 1 x T x N_classes
    labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
    end = datetime.datetime.now() - start

target_labels = list(dataset.target_vocab.keys())
print([rus[target_labels[l]] for l in labels])
print(end)

['местоимение', 'глагол', 'наречие', 'послелог', 'определитель', 'прилагательное', 'существительное', 'союз сочинительный', 'глагол', 'местоимение']
0:00:00.002128


In [56]:
#example 2
phrase = 'My mother seldom watches TV in the living-room , because she has little time'
words = phrase.split(' ')
tokens = [dataset.word_vocab[w] for w in words]

start = datetime.datetime.now()
with torch.no_grad():
    model_LSTM.eval()
    predict = model_LSTM(torch.tensor(tokens).unsqueeze(0).to(device)) # 1 x T x N_classes
    labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
    end = datetime.datetime.now() - start

target_labels = list(dataset.target_vocab.keys())
print([rus[target_labels[l]] for l in labels])
print(end)

['местоимение', 'существительное', 'наречие', 'существительное', 'существительное', 'послелог', 'определитель', 'существительное', 'пунктуация', 'союз подчинительный', 'местоимение', 'вспомогательный глагол', 'прилагательное', 'существительное']
0:00:00.001998


# GRU

In [57]:
class GRUPredictor(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, n_classes):
        super().__init__()
        #TODO try to use other RNN archicetures, f.e. RNN and LSTM
        self.word_emb = nn.Embedding(vocab_size, emb_dim)
        self.rnn = nn.GRU(emb_dim, hidden_dim, batch_first=True)
        self.clf = nn.Linear(hidden_dim, n_classes)
        self.do = nn.Dropout(0.1)

    def forward(self, x):
        emb = self.word_emb(x) # B x T x Emb_dim
        hidden, _ = self.rnn(emb) # B x T x Hid, B x 1 x Hid
        pred = self.clf(self.do(hidden)) # B x T x N_classes

        return pred

In [58]:
model_GRU = GRUPredictor(vocab_size, emb_dim, hidden, n_classes).to(device)
model_GRU.train()
optim = torch.optim.Adam(model_GRU.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

In [59]:
start = datetime.datetime.now()
for epoch in range(n_epochs):
    dataloader = DataLoader(dataset,
                            batch_size,
                            shuffle=True,
                            collate_fn=collate_fn,
                            drop_last = True,
                            )
    for i, batch in enumerate(dataloader):
        optim.zero_grad()

        predict = model_GRU(batch['data'].to(device))
        loss = loss_func(predict.view(-1, n_classes),
                         batch['target'].to(device).view(-1),
                         )
        loss.backward()
        optim.step()
        if i % 100 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')

    torch.save(model_GRU.state_dict(), f'./data/chkpt/gru_chkpt_{epoch}.pth')
end = datetime.datetime.now() - start
print(end)

epoch: 0, step: 0, loss: 2.96183443069458
epoch: 0, step: 100, loss: 0.2988634407520294
epoch: 0, step: 200, loss: 0.17352457344532013
epoch: 1, step: 0, loss: 0.19690696895122528
epoch: 1, step: 100, loss: 0.1286381036043167
epoch: 1, step: 200, loss: 0.12719474732875824
epoch: 2, step: 0, loss: 0.16245992481708527
epoch: 2, step: 100, loss: 0.09611711651086807
epoch: 2, step: 200, loss: 0.12764838337898254
epoch: 3, step: 0, loss: 0.060830000787973404
epoch: 3, step: 100, loss: 0.12090741097927094
epoch: 3, step: 200, loss: 0.11750597506761551
epoch: 4, step: 0, loss: 0.07578935474157333
epoch: 4, step: 100, loss: 0.065404511988163
epoch: 4, step: 200, loss: 0.07084160298109055
epoch: 5, step: 0, loss: 0.07452517002820969
epoch: 5, step: 100, loss: 0.03176399692893028
epoch: 5, step: 200, loss: 0.04402024671435356
epoch: 6, step: 0, loss: 0.06112167611718178
epoch: 6, step: 100, loss: 0.04922901839017868
epoch: 6, step: 200, loss: 0.032067298889160156
epoch: 7, step: 0, loss: 0.02813

In [60]:
#example
phrase = 'He ran quickly after the red bus and caught it'
words = phrase.split(' ')
tokens = [dataset.word_vocab[w] for w in words]

start = datetime.datetime.now()
with torch.no_grad():
    model_GRU.eval()
    predict = model_GRU(torch.tensor(tokens).unsqueeze(0).to(device)) # 1 x T x N_classes
    labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
    end = datetime.datetime.now() - start

target_labels = list(dataset.target_vocab.keys())
print([rus[target_labels[l]] for l in labels])
print(end)

['местоимение', 'глагол', 'наречие', 'послелог', 'определитель', 'прилагательное', 'существительное', 'союз сочинительный', 'глагол', 'местоимение']
0:00:00.003997


In [61]:
#example 2
phrase = 'My mother seldom watches TV in the living-room , because she has little time'
words = phrase.split(' ')
tokens = [dataset.word_vocab[w] for w in words]

start = datetime.datetime.now()
with torch.no_grad():
    model_GRU.eval()
    predict = model_GRU(torch.tensor(tokens).unsqueeze(0).to(device)) # 1 x T x N_classes
    labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
    end = datetime.datetime.now() - start

target_labels = list(dataset.target_vocab.keys())
print([rus[target_labels[l]] for l in labels])
print(end)

['местоимение', 'существительное', 'наречие', 'глагол', 'существительное', 'послелог', 'определитель', 'существительное', 'пунктуация', 'союз подчинительный', 'местоимение', 'вспомогательный глагол', 'прилагательное', 'существительное']
0:00:00.002999


In [79]:
import re
phrase = r"I live in a house near the mountains. I have two brothers and one sister, and I was born last. My father teaches mathematics, and my mother is a nurse at a big hospital. My brothers are very smart and work hard in school. My sister is a nervous girl, but she is very kind. My grandmother also lives with us. She came from Italy when I was two years old. She has grown old, but she is still very strong."
phrase = re.sub('([.,])', r' \1', phrase)
words = phrase.split(' ')
tokens = [dataset.word_vocab[w] for w in words]

start = datetime.datetime.now()
with torch.no_grad():
    model_RNN.eval()
    predict = model_RNN(torch.tensor(tokens).unsqueeze(0).to(device)) # 1 x T x N_classes
    labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
    end = datetime.datetime.now() - start

target_labels = list(dataset.target_vocab.keys())
RNN_pred = [rus[target_labels[l]] for l in labels]
print(f'Результат RNN модели:\n {RNN_pred}')
print(f'Время предсказания: {end}')

start = datetime.datetime.now()
with torch.no_grad():
    model_LSTM.eval()
    predict = model_LSTM(torch.tensor(tokens).unsqueeze(0).to(device)) # 1 x T x N_classes
    labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
    end = datetime.datetime.now() - start

target_labels = list(dataset.target_vocab.keys())
LSTM_pred = [rus[target_labels[l]] for l in labels]
print(f'Результат LSTM модели:\n {LSTM_pred}')
print(f'Время предсказания: {end}')

start = datetime.datetime.now()
with torch.no_grad():
    model_GRU.eval()
    predict = model_GRU(torch.tensor(tokens).unsqueeze(0).to(device)) # 1 x T x N_classes
    labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
    end = datetime.datetime.now() - start

target_labels = list(dataset.target_vocab.keys())
GRU_pred = [rus[target_labels[l]] for l in labels]
print(f'Результат GRU модели:\n {GRU_pred}')
print(f'Время предсказания: {end}')

Результат RNN модели:
 ['местоимение', 'глагол', 'послелог', 'определитель', 'существительное', 'послелог', 'определитель', 'существительное', 'пунктуация', 'местоимение', 'вспомогательный глагол', 'числительное', 'существительное', 'союз сочинительный', 'числительное', 'существительное', 'пунктуация', 'союз сочинительный', 'местоимение', 'вспомогательный глагол', 'глагол', 'прилагательное', 'пунктуация', 'местоимение', 'существительное', 'имя собственное', 'существительное', 'пунктуация', 'союз сочинительный', 'местоимение', 'существительное', 'вспомогательный глагол', 'определитель', 'существительное', 'послелог', 'определитель', 'прилагательное', 'существительное', 'пунктуация', 'местоимение', 'существительное', 'вспомогательный глагол', 'наречие', 'прилагательное', 'союз сочинительный', 'глагол', 'наречие', 'послелог', 'существительное', 'пунктуация', 'местоимение', 'существительное', 'вспомогательный глагол', 'определитель', 'прилагательное', 'существительное', 'пунктуация', 'союз

In [80]:
RNN_pred == LSTM_pred

False

In [81]:
RNN_pred == GRU_pred

False

In [83]:
GRU_pred == LSTM_pred

True

In [82]:
for i, pred in enumerate(RNN_pred):
    if pred != LSTM_pred[i]:
        print(f'Слово: {words[i]} \n'
              f'предсказание RNN: {pred} \n'
              f'предсказание LSTM: {LSTM_pred[i]}\n'
              f'предсказание GRU: {GRU_pred[i]}')

Слово: teaches 
предсказание RNN: имя собственное 
предсказание LSTM: глагол
предсказание GRU: глагол
Слово: when 
предсказание RNN: союз подчинительный 
предсказание LSTM: наречие
предсказание GRU: наречие


**Вывод:** самая быстрая в обучении модель RNN, она же самая быстрая в предсказании. На тестовом примере LSTM и GRU показали одинаковые результаты, при этом, GRU учится быстрее. Сеть на базе RNN допустила 2 ошибки

# RNN с bidirectional

In [84]:
class RNNPredictor_bi_dir(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, n_classes):
        super().__init__()
        self.word_emb = nn.Embedding(vocab_size, emb_dim)
        #TODO try to use other RNN archicetures, f.e. RNN and LSTM
        self.rnn = nn.RNN(emb_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.clf = nn.Linear(hidden_dim*2, n_classes)
        self.do = nn.Dropout(0.1)

    def forward(self, x):
        emb = self.word_emb(x) # B x T x Emb_dim
        hidden, _ = self.rnn(emb) # B x T x Hid, B x 1 x Hid
        hidden = self.do(hidden)
        pred = self.clf(hidden) # B x T x N_classes

        return pred

In [85]:
model_RNN_bi_dir = RNNPredictor_bi_dir(vocab_size, emb_dim, hidden, n_classes).to(device)
model_RNN_bi_dir.train()
optim = torch.optim.Adam(model_RNN_bi_dir.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

In [86]:
start = datetime.datetime.now()
for epoch in range(n_epochs):
    dataloader = DataLoader(dataset,
                            batch_size,
                            shuffle=True,
                            collate_fn=collate_fn,
                            drop_last = True,
                            )
    for i, batch in enumerate(dataloader):
        optim.zero_grad()

        predict = model_RNN_bi_dir(batch['data'].to(device))
        loss = loss_func(predict.view(-1, n_classes),
                         batch['target'].to(device).view(-1),
                         )
        loss.backward()
        optim.step()
        if i % 100 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')

    torch.save(model_RNN_bi_dir.state_dict(), f'./data/chkpt/rnn_bi_dir_chkpt_{epoch}.pth')
end = datetime.datetime.now() - start
print(end)

epoch: 0, step: 0, loss: 3.1407840251922607
epoch: 0, step: 100, loss: 0.20730912685394287
epoch: 0, step: 200, loss: 0.18161946535110474
epoch: 1, step: 0, loss: 0.21676044166088104
epoch: 1, step: 100, loss: 0.11265759915113449
epoch: 1, step: 200, loss: 0.11493123322725296
epoch: 2, step: 0, loss: 0.0695004090666771
epoch: 2, step: 100, loss: 0.12229912728071213
epoch: 2, step: 200, loss: 0.062385689467191696
epoch: 3, step: 0, loss: 0.06956055760383606
epoch: 3, step: 100, loss: 0.05425114184617996
epoch: 3, step: 200, loss: 0.07239409536123276
epoch: 4, step: 0, loss: 0.0769365131855011
epoch: 4, step: 100, loss: 0.04254380613565445
epoch: 4, step: 200, loss: 0.04885401204228401
epoch: 5, step: 0, loss: 0.054897937923669815
epoch: 5, step: 100, loss: 0.04962540417909622
epoch: 5, step: 200, loss: 0.03757314011454582
epoch: 6, step: 0, loss: 0.03546896204352379
epoch: 6, step: 100, loss: 0.03118445724248886
epoch: 6, step: 200, loss: 0.03003990463912487
epoch: 7, step: 0, loss: 0.0

# LSTM c bidirectional

In [95]:
class LSTMpredictor_bi_dir(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, n_classes):
        super().__init__()
        self.word_emb = nn.Embedding(vocab_size, emb_dim)
        #TODO try to use other RNN archicetures, f.e. RNN and LSTM
        self.rnn = nn.LSTM(emb_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.clf = nn.Linear(hidden_dim*2, n_classes)
        self.do = nn.Dropout(0.1)

    def forward(self, x):
        emb = self.word_emb(x) # B x T x Emb_dim
        hidden, _ = self.rnn(emb) # B x T x Hid, B x 1 x Hid
        hidden = self.do(hidden)
        pred = self.clf(hidden) # B x T x N_classes

        return pred

In [96]:
model_LSTM_bi_dir = LSTMpredictor_bi_dir(vocab_size, emb_dim, hidden, n_classes).to(device)
model_LSTM_bi_dir.train()
optim = torch.optim.Adam(model_LSTM_bi_dir.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

In [97]:
start = datetime.datetime.now()
for epoch in range(n_epochs):
    dataloader = DataLoader(dataset,
                            batch_size,
                            shuffle=True,
                            collate_fn=collate_fn,
                            drop_last=True,
                            )
    for i, batch in enumerate(dataloader):
        optim.zero_grad()

        predict = model_LSTM_bi_dir(batch['data'].to(device))
        loss = loss_func(predict.view(-1, n_classes),
                         batch['target'].to(device).view(-1),
                         )
        loss.backward()
        optim.step()
        if i % 100 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')

    torch.save(model_LSTM_bi_dir.state_dict(), f'./data/chkpt/lstm_chkpt_{epoch}.pth')
end = datetime.datetime.now() - start
print(end)

epoch: 0, step: 0, loss: 2.925898790359497
epoch: 0, step: 100, loss: 0.21059846878051758
epoch: 0, step: 200, loss: 0.1369604617357254
epoch: 1, step: 0, loss: 0.1001410186290741
epoch: 1, step: 100, loss: 0.14304573833942413
epoch: 1, step: 200, loss: 0.12774835526943207
epoch: 2, step: 0, loss: 0.08160749822854996
epoch: 2, step: 100, loss: 0.06445468217134476
epoch: 2, step: 200, loss: 0.05662434920668602
epoch: 3, step: 0, loss: 0.05439642071723938
epoch: 3, step: 100, loss: 0.05366320535540581
epoch: 3, step: 200, loss: 0.05912519991397858
epoch: 4, step: 0, loss: 0.035853032022714615
epoch: 4, step: 100, loss: 0.04357300326228142
epoch: 4, step: 200, loss: 0.0528816320002079
epoch: 5, step: 0, loss: 0.024363698437809944
epoch: 5, step: 100, loss: 0.030885355547070503
epoch: 5, step: 200, loss: 0.03241448476910591
epoch: 6, step: 0, loss: 0.02153574489057064
epoch: 6, step: 100, loss: 0.018643667921423912
epoch: 6, step: 200, loss: 0.013470453210175037
epoch: 7, step: 0, loss: 0.

# GRU c bidirectional

In [98]:
class GRUPredictor_bi_dir(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, n_classes):
        super().__init__()
        #TODO try to use other RNN archicetures, f.e. RNN and LSTM
        self.word_emb = nn.Embedding(vocab_size, emb_dim)
        self.rnn = nn.GRU(emb_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.clf = nn.Linear(hidden_dim*2, n_classes)
        self.do = nn.Dropout(0.1)

    def forward(self, x):
        emb = self.word_emb(x)  # B x T x Emb_dim
        hidden, _ = self.rnn(emb)  # B x T x Hid, B x 1 x Hid
        pred = self.clf(self.do(hidden))  # B x T x N_classes

        return pred

In [99]:
model_GRU_bi_dir = GRUPredictor_bi_dir(vocab_size, emb_dim, hidden, n_classes).to(device)
model_GRU_bi_dir.train()
optim = torch.optim.Adam(model_GRU_bi_dir.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

In [100]:
start = datetime.datetime.now()
for epoch in range(n_epochs):
    dataloader = DataLoader(dataset,
                            batch_size,
                            shuffle=True,
                            collate_fn=collate_fn,
                            drop_last=True,
                            )
    for i, batch in enumerate(dataloader):
        optim.zero_grad()

        predict = model_GRU_bi_dir(batch['data'].to(device))
        loss = loss_func(predict.view(-1, n_classes),
                         batch['target'].to(device).view(-1),
                         )
        loss.backward()
        optim.step()
        if i % 100 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')

    torch.save(model_GRU_bi_dir.state_dict(), f'./data/chkpt/gru_chkpt_{epoch}.pth')
end = datetime.datetime.now() - start
print(end)

epoch: 0, step: 0, loss: 2.8381030559539795
epoch: 0, step: 100, loss: 0.22998511791229248
epoch: 0, step: 200, loss: 0.20958945155143738
epoch: 1, step: 0, loss: 0.14291366934776306
epoch: 1, step: 100, loss: 0.11516620218753815
epoch: 1, step: 200, loss: 0.09671066701412201
epoch: 2, step: 0, loss: 0.0654531717300415
epoch: 2, step: 100, loss: 0.0889594629406929
epoch: 2, step: 200, loss: 0.06926024705171585
epoch: 3, step: 0, loss: 0.05963835120201111
epoch: 3, step: 100, loss: 0.06701663881540298
epoch: 3, step: 200, loss: 0.060355450958013535
epoch: 4, step: 0, loss: 0.037076883018016815
epoch: 4, step: 100, loss: 0.05435841530561447
epoch: 4, step: 200, loss: 0.042370568960905075
epoch: 5, step: 0, loss: 0.03276119381189346
epoch: 5, step: 100, loss: 0.029102355241775513
epoch: 5, step: 200, loss: 0.03584596887230873
epoch: 6, step: 0, loss: 0.021334558725357056
epoch: 6, step: 100, loss: 0.02521331049501896
epoch: 6, step: 200, loss: 0.02247072197496891
epoch: 7, step: 0, loss: 

# Сравнение

In [101]:
start = datetime.datetime.now()
with torch.no_grad():
    model_RNN_bi_dir.eval()
    predict = model_RNN_bi_dir(torch.tensor(tokens).unsqueeze(0).to(device)) # 1 x T x N_classes
    labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
    end = datetime.datetime.now() - start

target_labels = list(dataset.target_vocab.keys())
RNN_pred_bi_dir = [rus[target_labels[l]] for l in labels]
print(f'Результат RNN модели c bidirectional:\n {RNN_pred_bi_dir}')
print(f'Время предсказания: {end}')

start = datetime.datetime.now()
with torch.no_grad():
    model_LSTM_bi_dir.eval()
    predict = model_LSTM_bi_dir(torch.tensor(tokens).unsqueeze(0).to(device)) # 1 x T x N_classes
    labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
    end = datetime.datetime.now() - start

target_labels = list(dataset.target_vocab.keys())
LSTM_pred_bi_dir = [rus[target_labels[l]] for l in labels]
print(f'Результат LSTM модели c bidirectional:\n {LSTM_pred_bi_dir}')
print(f'Время предсказания: {end}')

start = datetime.datetime.now()
with torch.no_grad():
    model_GRU_bi_dir.eval()
    predict = model_GRU_bi_dir(torch.tensor(tokens).unsqueeze(0).to(device)) # 1 x T x N_classes
    labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
    end = datetime.datetime.now() - start

target_labels = list(dataset.target_vocab.keys())
GRU_pred_bi_dir = [rus[target_labels[l]] for l in labels]
print(f'Результат GRU модели c bidirectional:\n {GRU_pred_bi_dir}')
print(f'Время предсказания: {end}')

Результат RNN модели c bidirectional:
 ['местоимение', 'глагол', 'послелог', 'определитель', 'существительное', 'послелог', 'определитель', 'существительное', 'пунктуация', 'местоимение', 'глагол', 'числительное', 'существительное', 'союз сочинительный', 'числительное', 'существительное', 'пунктуация', 'союз сочинительный', 'местоимение', 'вспомогательный глагол', 'глагол', 'прилагательное', 'пунктуация', 'местоимение', 'существительное', 'глагол', 'существительное', 'пунктуация', 'союз сочинительный', 'местоимение', 'существительное', 'вспомогательный глагол', 'определитель', 'существительное', 'послелог', 'определитель', 'прилагательное', 'существительное', 'пунктуация', 'местоимение', 'существительное', 'вспомогательный глагол', 'наречие', 'прилагательное', 'союз сочинительный', 'глагол', 'наречие', 'послелог', 'существительное', 'пунктуация', 'местоимение', 'существительное', 'вспомогательный глагол', 'определитель', 'прилагательное', 'существительное', 'пунктуация', 'союз сочините

In [102]:
RNN_pred_bi_dir == RNN_pred

False

In [103]:
LSTM_pred_bi_dir == LSTM_pred

False

In [104]:
GRU_pred_bi_dir == GRU_pred

False

In [106]:
for i, pred in enumerate(RNN_pred_bi_dir):
    if pred != LSTM_pred[i]:
        print(f'Слово: {words[i]} \n'
              f'предсказание RNN c bidirectional: {pred} \n'
              f'предсказание RNN: {RNN_pred[i]} \n'
              f'предсказание LSTM c bidirectional: {LSTM_pred_bi_dir[i]}\n'
              f'предсказание LSTM: {LSTM_pred[i]}\n'
              f'предсказание GRU c bidirectional: {GRU_pred_bi_dir[i]}\n'
              f'предсказание GRU: {GRU_pred[i]}')

Слово: have 
предсказание RNN c bidirectional: глагол 
предсказание RNN: вспомогательный глагол 
предсказание LSTM c bidirectional: глагол
предсказание LSTM: вспомогательный глагол
предсказание GRU c bidirectional: глагол
предсказание GRU: вспомогательный глагол
Слово: kind 
предсказание RNN c bidirectional: наречие 
предсказание RNN: прилагательное 
предсказание LSTM c bidirectional: прилагательное
предсказание LSTM: прилагательное
предсказание GRU c bidirectional: прилагательное
предсказание GRU: прилагательное
Слово: when 
предсказание RNN c bidirectional: союз подчинительный 
предсказание RNN: союз подчинительный 
предсказание LSTM c bidirectional: наречие
предсказание LSTM: наречие
предсказание GRU c bidirectional: наречие
предсказание GRU: наречие


In [107]:
for i, pred in enumerate(RNN_pred_bi_dir):
    if pred != RNN_pred[i]:
        print(f'Слово: {words[i]} \n'
              f'предсказание RNN c bidirectional: {pred} \n'
              f'предсказание RNN: {RNN_pred[i]} \n')

Слово: have 
предсказание RNN c bidirectional: глагол 
предсказание RNN: вспомогательный глагол 

Слово: teaches 
предсказание RNN c bidirectional: глагол 
предсказание RNN: имя собственное 

Слово: kind 
предсказание RNN c bidirectional: наречие 
предсказание RNN: прилагательное 



In [108]:
for i, pred in enumerate(LSTM_pred_bi_dir):
    if pred != LSTM_pred[i]:
        print(f'Слово: {words[i]} \n'
              f'предсказание LSTM c bidirectional: {pred} \n'
              f'предсказание LSTM: {LSTM_pred[i]} \n')

Слово: have 
предсказание LSTM c bidirectional: глагол 
предсказание LSTM: вспомогательный глагол 

Слово: teaches 
предсказание LSTM c bidirectional: существительное 
предсказание LSTM: глагол 



In [109]:
for i, pred in enumerate(GRU_pred_bi_dir):
    if pred != GRU_pred[i]:
        print(f'Слово: {words[i]} \n'
              f'предсказание GRU c bidirectional: {pred} \n'
              f'предсказание GRU: {GRU_pred[i]} \n')

Слово: have 
предсказание GRU c bidirectional: глагол 
предсказание GRU: вспомогательный глагол 

Слово: teaches 
предсказание GRU c bidirectional: послелог 
предсказание GRU: глагол 



**Вывод:** модели с bidirectional отработали хуже и ошиблись на тех словах, в которых ранее ошибок не было. Также, такие модели учатся и предсказывают значительно медленнее